In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

We follow the procedure outlined in 3.1.2.1 (Experiment 1)

Step 1: Generation of $\underline{\beta}, \underline{\gamma} \in \mathbb{R}^p$

In [2]:
# TODO: Split up optimiser into satisfy and assigment weight

In [3]:
from benchmark.random_k_sat import RandomKSAT

k = 3
n = 8
num_instances = 10

# TODO: Generate 100 random problem instances from CNF(12, 8, r_k)
instances = [RandomKSAT.from_poisson(n, k).formula for _ in range(num_instances)]

In [3]:
from k_sat.quantum_solver.pauli_encoder import PauliEncoder
from k_sat.quantum_solver.average_optimiser import AverageOptimiser

ps = [1, 2] #, 4, 8, 30, 60]
optimal_params = {}

encoder = PauliEncoder()
optimiser = AverageOptimiser()

for p in ps:

	# Encode instances
	training_circuits = [(formula, encoder.encode_formula(formula, p)) for formula in instances]

	# Initialise gamma_i = -0.01, beta_i = 0.01 
	initial_params = [-0.01, 0.01] * p

	# Find and store optimal paramters across instances
	optimal_params[p] = optimiser.find_optimal_params(initial_params, training_circuits)

In [4]:
optimal_params

{1: array([0.70722667, 0.41313836]),
 2: array([ 0.49547756, -0.31445658,  0.99037807, -0.53995839])}

In [5]:
from k_sat.quantum_solver.qaoa_evaluator import QAOAEvaluator

p_succ = {}
running_times = {}

# TODO: Evaluate parameters on 10000 random satisfiable instances
for (p, params) in optimal_params.items():
	formula = RandomKSAT.from_poisson(n, k).formula
	circuit = encoder.encode_formula(formula, p)
	evaluator = QAOAEvaluator(circuit, formula, params)
	p_succ[p] = evaluator.success_probability()
	running_times[p] = evaluator.running_time(timeout=200)


Samples drawn: 10


In [6]:
p_succ

{1: 0.11584136683334464, 2: 0.12523635976988026}

In [7]:
running_times

{1: ('01111110', 2), 2: ('00110111', 19)}